# Conventional Neural Network

### Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.13.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [ ]:
dataset = pd.read_csv('data.txt',sep="\t", header=0)
X = dataset.iloc[:, 2:8].values
y = [0 for i in range(len(X[:,0]))]


In [ ]:
T_th = 10000 #Temperture increasement threshold
Pd_th = 100 #Proton density increasement threshold

for i in range(1,len(y)):
  if X[i,5] > X[i-1,5] + T_th: #Temperture
    if X[i,4] > X[i-1,4] + Pd_th: #Proton density
      if (round(X[i,2],1) == round(X[i-1,2],1)) or ((round(X[i,1],1) == round(X[i-1,1],1))): #Bz and By
        y[i] = 1

### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

## Part 2 - Building the CNN

### Initializing the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

# Add the input layer (reshape input data to match the expected input shape of CNN)


In [ ]:
cnn.add(tf.keras.layers.Reshape((6, 1, 1), input_shape=(6,)))


### Add convolutional layers


In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 1), activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 1)))

### Adding the third hidden layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 1), activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 1)))

# Flatten the output before fully connected layers


In [ ]:
cnn.add(tf.keras.layers.Flatten())


### Add fully connected layers


In [ ]:
cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001, clipnorm=1.0)
cnn.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])


### Training the CNN on the Training set

In [ ]:
cnn.fit(X_train, y_train, batch_size = 32, epochs = 100)

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Solution**

In [ ]:
single_observation = np.array([[-1, 0.2, 0.2, 0.4, 0.5, 1]])
prediction = cnn.predict(single_observation)
print(prediction > 0.5)

### Predicting the Test set results

In [ ]:
y_pred = cnn.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

### Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)